<a href="https://colab.research.google.com/github/RyuMyunggi/design-pattern/blob/main/observer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 옵져버 패턴

## 행위 패턴

* 행위패턴은 이름 그대로 객체의 역할에 초점을 둠
* 더 큰 기능을 구현하기 위한 객체 간의 상호 작용을 중요시함
* 행위 패턴에서의 객체는 상호 작용하지만 느슨하게 결합돼 있음
* 옵져버 디자인 패턴은 가장 단순한 행위 패턴

### 옵져버 패턴
* 옵져버 패턴에서 객체(서브젝트)는 자식(옵져버)의 목록을 유지하며 서브젝트가 옵져버에 정의된 메소드를 호출 할 때 마다 옵져버에게 이를 알림
* 애플리케이션을 구성하는 여러 서비스를 관리하는 코어 서비스는 옵져버의 상태를 모니터링하는 서브젝트임
* 옵져버는 서브젝트의 상태에 따라 자신의 객체 상태를 변경하거나 필요한 연산을 수행
* 이처럼 종속된 서비스가 코어 서비스의 상태를 참고하는 구조에서는 옵져버 디자인 패턴이 적합


### 옵져버 패턴의 목적
* 객체 간 일대다 관계를 형성하고 객체의 상태를 다른 종속 객체에 자동으로 알림
* 서브젝트의 핵심 부분을 캡슐화함
* 분산 시스템의 이벤트 서비스를 구현할 때

## 옵져버 패턴의 구성원

1. Subject
* Subject는 Observer를 관리함
* Observer는 Subject 클래스의 register()와 deregister() 매소드를 호출해 자신을 등록
* Subject는 여러 옵져버들을 관리

2. Observer
* 서브젝트를 감시하는 객체를 위한 인터페이스를 제공
* 서브젝트의 상태르 알 수 있도록 ConcreaterObserver가 구현해야하는 메소드를 정의

3. ConcreateObserver
* Subject의 상태를 저장
* 서브젝트에 대한 정보와 실제 상태를 일관되게 유지하기 위해 Observer 인터페이스를 구현

ConcreateObserver는 Observer 인터페이스를구현해 자신을 Subject에 등록.
상태 변화가 있을 때마다 Subject는 Observer의 알림 메소드를 통해 모든 ConcreateObserver에 알림

In [8]:
# 기본 옵져버 패턴 구현

class Subjcet(object):
  def __init__(self):
    self.__observers = []

  def register(self, observer):
    self.__observers.append(observer)
  
  def notifyAll(self, *args, **kwargs):
    for observer in self.__observers:
      observer.notify(self, *args, **kwargs)


class Observer1(object):
  def __init__(self, subject):
    subject.register(self)

  def notify(self, subject, *args):
    print(type(self).__name__, ':: Got', args, 'From', subject)


class Observer2(object):
  def __init__(self, subject):
    subject.register(self)

  def notify(self, subject, *args):
    print(type(self).__name__, ':: Got', args, 'From', subject)


subject = Subjcet()
observer1 = Observer1(subject)
observer2 = Observer2(subject)

subject.notifyAll('notifycation')

Observer1 :: Got ('notifycation',) From <__main__.Subjcet object at 0x7f6427eda690>
Observer2 :: Got ('notifycation',) From <__main__.Subjcet object at 0x7f6427eda690>


In [22]:
# 뉴스 에이전시 옵져버 패턴 구현

class NewPublisher:
    def __init__(self):
        self.__subscribers = []
        self.__latesNews = None

    def attach(self, subscriber):
        self.__subscribers.append(subscriber)

    def detach(self):
        return self.__subscribers.pop()

    def subscribers(self):
        return [type(x).__name__ for x in self.__subscribers]
    
    def notifySubscribers(self):
        for sub in self.__subscribers:
            sub.update()

    def addNews(self, news):
        self.__latesNews = news
    
    def getNews(self):
        return 'Got News: ', self.__latesNews


In [10]:
# Subscriber
# 모든 ConcreateObserver의 추상 기본 클래스

from abc import ABCMeta
from abc import abstractmethod


class Subscriber(metaclass=ABCMeta):
    @abstractmethod
    def update(self):
      pass

In [23]:
# ConcreateObserver
# Subscriber 인터페이스를 구현하는 옵져버


class SMSSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())


class EmailSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())


class AnyOtherSubscriber:
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())


In [24]:
news_publisher = NewPublisher()
for Subscriber in [SMSSubscriber, EmailSubscriber, AnyOtherSubscriber]:
    Subscriber(news_publisher)
    
print('\nSubscriber: ', news_publisher.subscribers())

news_publisher.addNews('Hello World')
news_publisher.notifySubscribers()

print('\nDetached: ', type(news_publisher.detach()).__name__)
print('\nSubscriber: ', news_publisher.subscribers())

news_publisher.addNews('My Second News!')
news_publisher.notifySubscribers()


Subscriber:  ['SMSSubscriber', 'EmailSubscriber', 'AnyOtherSubscriber']
SMSSubscriber ('Got News: ', 'Hello World')
EmailSubscriber ('Got News: ', 'Hello World')
AnyOtherSubscriber ('Got News: ', 'Hello World')

Detached:  AnyOtherSubscriber

Subscriber:  ['SMSSubscriber', 'EmailSubscriber']
SMSSubscriber ('Got News: ', 'My Second News!')
EmailSubscriber ('Got News: ', 'My Second News!')


## 옵져버 패턴 패턴 매소드

* Subject의 변경 사항을 Observer에 알리는 방법에는 push / pull 두 가지 방법이 있음

### PULL 모델
* 풀 모델에서의 Observer는 아래의 역할을 함
   * Subject는 변경 사항이 있음을 등록된 Observer에 브로드캐스트 함
   * Observer는 직접 게시자에게 변경 사항을 요청하고 끌어와야함
   * 풀 모델은 Subject가 Observer에 알리는 단계와 Observer가 Subject로 부터 필요한 데이터를 받아오는 두 단계가 필요하므로 비효율적임

### PUSH 모델
  * 푸쉬 모델에서의 Observer는 아래의 역할을 함
    * 풀 모델과 달리 Subject가 Observer에 데이터를 보호함
    * Subject는 Observer가 필요로 하지 않는 데이터까지 보낼 수 있음
    * 따라서 쓸데 없이 많은 양의 데이터를 전송해 응답 시간이 늦어질 수 있음
    * 성능을 위해 Subject는 오직 필요로 한 데이터만 보내야함

## 느슨한 결합과 옵져버 패턴
* 느슨한 결합은 중요한 소프트웨어 애플리케이션 설계 원칙임
* 상호작용하는 객체 간의 관계를 최대한 느슨하게 구성하는 것이 목적
* 여기서의 결합이란 객체가 상호작용하는 다른 객체에 대해 알고있는 정도를 의미

### 느슨한 결합의 효과
* 한 부분에 대한 수정이 예기치 않은 다른 부분까지 영향을 미치는 위험을 줄임
* 테스트와 유지 보수 및 장애 처리가 쉽다
* 시스템을 쉽게 여러 부분으로 분리

### 옵져버 패턴과 느슨한 결합
* Subject는 정확히 Observer가 어떤 인터페이스를 구현하는지 모름. ConcreateObserver의 존재를 모름
* 언제든지 새로운 Observer를 추가 할 수 있음
* 새로운 Object를 추가해도 Subject를 수정할 필요가 없음. 위의 예제에서 Subject를 수정하지 않고도 AnyOtherObserver를 추가/제거 했음
* Subject 또는 Observer는 독립적임. Observer는 필요 시 어디에서도 재 사용될 수 있음
* Subjcet 또는 Observer에 대한 수정이 서로에게 아무런 영향을 주지 않음. 완전 독립성 또는 느슨한 결합 덕분에 걱정 없이 수정할 수 있음
* Subject 또는 Observer 클래스를 수정하지 않고 객체간 자유롭게 데이터를 주고 받을 수 있음
* 새로운 Observer를 언제든지 추가/제거할 수 있음

## 옵져버 패턴의 단점

* ConcreateObserver는 상속을 통해 Observer 인터페이스를 구현. 컴포지션에 대한 선택권이 없음
* 제대로 구현되지 안흔 Observer 클래스는 복잡도를 높이고 성능 저하의 원인이 될 수 있음
* 애플리케이션 알림 기능은 간혹 신뢰할 수 없으며 레이스 상태 또는 비일관성을 초래 할 수 있음